# Prerequisites

If you were running this on your own machine you would have
to install jupyter, neuroconv package and the data.

However for the workshop these steps have already been done
for you so you don't need to install anything.

# NWB workshop

## Converting to NWB with neuroconv

We will deal with three problems that we think arise often.
The first one is having a type of data that you would like
to convert to NWB format.  The second problem is that you already 
have an NWB file and some data you would like to add to the NWB file.
The third problem is that you have an NWB file and you would like to
extract some data from the file.


### Problem 1a:
We will start with the simplest case scenario.
We have a single source of data. The data that we have
are from a csv file, 
to convert them to NWB we will use the `CsvTimeIntervalsInterface`.

In [ ]:
# Import requirement for the conversion
from neuroconv.datainterfaces import CsvTimeIntervalsInterface
from datetime import datetime
from zoneinfo import ZoneInfo
from pathlib import Path

#### Create CSV file if not available

In [ ]:
import random
import pandas as pd
data_dir = Path("../data") # Data path
data_dir.mkdir(exist_ok=True) # Create data dir
csv_file_path = data_dir / "mydata.csv" # Form file data path
random.seed(42) # set seed
if not csv_file_path.exists():
    # Create csv file if it doesn't exist
    n = 100
    starts  = [-1]*n
    ends  = [-1]*n
    vals = [-1]*n
    s0 = 0
    for i in range(n):
        starts[i]= s0 + random.randrange(5)
        ends[i] = starts[i] + random.randrange(1,50)
        vals[i] = random.uniform(0,1)
        s0 = ends[i]


    pd.DataFrame({'start_time':starts, 'end_time':ends, 'value':vals }).to_csv(csv_file_path,index=False)

#### Create interface

In [ ]:
csv_interface = CsvTimeIntervalsInterface(file_path=csv_file_path, verbose=False)

#### Get metadata

In [ ]:
metadata = csv_interface.get_metadata()
metadata

#### Attempt to create NWB file

In [ ]:
prob1a_nwb_file = data_dir/"problem_1a.nwb"
csv_interface.run_conversion(nwbfile_path = prob1a_nwb_file)

#### Add missing metadata

In [ ]:
session_start_time = datetime(2025,1,10,11,45,0, tzinfo=ZoneInfo("Europe/Paris"))
metadata["NWBFile"]["session_start_time"] = session_start_time

#### Create NWB file

In [ ]:
csv_interface.run_conversion(nwbfile_path = prob1a_nwb_file, metadata=metadata)

#### Look at the generated NWB file

In [ ]:
from pynwb import NWBHDF5IO
import pynwb

In [ ]:
# Ideally we would use a with statement 
# as in the commented code
#
## with NWBHDF5IO(prob1a_nwb_file, mode = 'r') as io:
##    nwbfile = io.read()
##    nwbfile
# we instead use the code below to be able to
# see a nice version of the file.
io = NWBHDF5IO(prob1a_nwb_file, mode = 'r')
nwbfile = io.read()
nwbfile

In [ ]:
io.close() # Don't forget to close the file

#### Use nwbwidgets to look at the file
Just a taste of how nwbwidgets work, we will take a closer look at nwbwidgets later on.

In [ ]:
from nwbwidgets import nwb2widget

io = NWBHDF5IO(prob1a_nwb_file, mode = 'r')
nwbfile = io.read()
nwb2widget(nwbfile)

In [ ]:
io.close()

###  Problem 1b
Now we again want to generate a NWBFile from already available data.
However, in addition to the csv file with some tiff images that 
should also be inside the generated NWB file.
#### Download tiff file

In [ ]:
%%bash
# Download movie file if not already available
if [[ ! -e "../data/demoMovie.tif" ]]; then
   wget https://github.com/flatironinstitute/CaImAn/raw/refs/heads/main/example_movies/demoMovie.tif -O ../data/demoMovie.tif
fi        

In [ ]:
from neuroconv.datainterfaces import TiffImagingInterface
from neuroconv import NWBConverter
movie_path = data_dir / 'demoMovie.tif'
prob1b_nwb_file = data_dir / 'problem_1b.nwb'


class MyConverter(NWBConverter):
    data_interface_classes = dict (
        csvIntervals = CsvTimeIntervalsInterface,
        movieRecording = TiffImagingInterface )

sourceData = dict(
      csvIntervals = dict(file_path=csv_file_path),
      movieRecording = dict(file_path=movie_path, sampling_frequency=15.0))

dual_converter = MyConverter(sourceData)

metadata = dual_converter.get_metadata()
metadata

In [ ]:
metadata["NWBFile"]["session_start_time"] = session_start_time
dual_converter.run_conversion(metadata=metadata, nwbfile_path=prob1b_nwb_file)

### Problem 2
In this problem we have an NWB file with some data (the file we
created in problem 1a) and we have acquired some new data the tiff
file from problem 1b). We want to have all the data in a single file.
We will use two approaches:
1. Append the data to an existing nwb file on disk.
2. Create a new nwb file in memory file and save it.

#### Create an appropriate interface
First we create an appropriate interface

In [ ]:
tiff_interface = TiffImagingInterface(file_path=movie_path, sampling_frequency=15.0)

#### Append data to an existing NWB file
We copy the file we created in problem 1a

In [ ]:
# First we create a copy of the file we created in  problem 1a
import shutil
prob2a_nwb_file = data_dir / "problem_2a.nwb"

shutil.copyfile(prob1a_nwb_file, prob2a_nwb_file)

In [ ]:
tiff_interface.run_conversion(prob2a_nwb_file)

#### Create an NWB file in memory and save it

In [ ]:
prob2b_nwb_file = data_dir / "problem_2b.nwb"
with NWBHDF5IO(prob1a_nwb_file, mode = 'r') as fin, NWBHDF5IO(prob2b_nwb_file, mode = 'w' ) as fout:
    prob1a = fin.read() # Read nwb file from prob1a
    tiff_interface.add_to_nwbfile(prob1a) # Add the photon information to prob1a, modifies in place
    fout.export(fin, nwbfile=prob1a) # Export the new file

### Problem 3

In this problem we are looking at the scenario where we have an NWB file already. 
However, we would like to remove some information and save the result as an NWB file.
We will start with the NWB file we created in problem 1b and remove the TwoPhotonSeries
from acquisition. Not you can pop items only from LabelledDict items.

In [ ]:
fin = NWBHDF5IO(prob1b_nwb_file, mode = 'r')
prob1b = fin.read()
prob1b

In [ ]:
type(prob1b.acquisition)

In [ ]:
two_photon = prob1b.acquisition.pop('TwoPhotonSeries')
prob1b

In [ ]:
prob3_nwb_file = data_dir / "problem_3.nwb"
with NWBHDF5IO(prob3_nwb_file, mode = 'w' ) as fout:
    fout.export(fin, nwbfile=prob1b) 

In [ ]:
fin.close()

## NWBwidgets
A closer look at NWB widgets. We will look at somefile from the DANDI archive. Select DANDI using the radio button. Then select dandiset 4, from the dataset we will look at the nwb file for sub-P27CS

In [ ]:
from nwbwidgets.panel import Panel
Panel()